In [113]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.linear_model import LinearRegression

In [114]:
data = pd.read_csv('train.csv') # Loading training data 

In [115]:
len(data.columns)

17

## Reducing Memory Usage

In [116]:
def reduce_memory_usage(df):
    print("Memory Usage Before Optimization:")
    print(df.memory_usage(deep=True).sum() / (1024 ** 2), "MB")

    # Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is numeric
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
                    
    # Display the memory usage after optimization
    print("\nMemory Usage After Optimization:")
    print(df.memory_usage(deep=True).sum() / (1024 ** 2), "MB")
    return df

data = reduce_memory_usage(data)

Memory Usage Before Optimization:
974.2033262252808 MB

Memory Usage After Optimization:
599.5538015365601 MB


In [117]:
data.head() # Reading first few examples 

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,60651.500000,1.000026,8493.030273,1.0,-3.029704,0,0_0_0
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.040039,1.000660,20605.089844,1.0,-5.519986,0,0_0_1
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,37956.000000,1.000298,18995.000000,1.0,-8.389950,0,0_0_2
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,2324.899902,1.000214,479032.406250,1.0,-4.010201,0,0_0_3
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849,0,0_0_4


## Rolling Averages for Missing Elements

In [118]:
print(data.isnull().sum()) # Displaying columns with missing data

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64


In [119]:
# Creating list of columns with missing data 
columns_with_missing_values = ['imbalance_size', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap', 'target']

In [120]:
# Checking if any columns start with missing values

first_missing_index_imbalance_size = data['imbalance_size'].isnull().idxmax() if data['imbalance_size'].isnull().any() else None
first_missing_index_reference_price = data['reference_price'].isnull().idxmax() if data['reference_price'].isnull().any() else None
first_missing_index_matched_size = data['matched_size'].isnull().idxmax() if data['matched_size'].isnull().any() else None
first_missing_index_far_price = data['far_price'].isnull().idxmax() if data['far_price'].isnull().any() else None
first_missing_index_near_price = data['near_price'].isnull().idxmax() if data['near_price'].isnull().any() else None
first_missing_index_bid_price = data['bid_price'].isnull().idxmax() if data['bid_price'].isnull().any() else None
first_missing_index_ask_price = data['ask_price'].isnull().idxmax() if data['ask_price'].isnull().any() else None
first_missing_index_wap = data['wap'].isnull().idxmax() if data['wap'].isnull().any() else None
first_missing_index_target = data['target'].isnull().idxmax() if data['target'].isnull().any() else None


first_missing_index_imbalance_size, first_missing_index_reference_price, first_missing_index_matched_size, first_missing_index_far_price, first_missing_index_near_price, first_missing_index_bid_price, first_missing_index_ask_price, first_missing_index_wap, first_missing_index_target

(369508, 369508, 369508, 0, 0, 369508, 369508, 369508, 369508)

## 'far_price' and 'near_price' columns start with missing values so we need to handle these edge cases using linear interpolation

In [121]:
window_size = 5 # Number of elements before current missing element to take an average of 

for col in columns_with_missing_values:
    # Create rolling mean for the columns

    data[col + '_rolling_mean'] = data[col].rolling(window = window_size, min_periods = 1).mean()

    data[col] = data[col].fillna(data[col + '_rolling_mean']) # Fill missing values in the original column with the rolling average

    data.drop(columns=[col + '_rolling_mean'], inplace=True) # Drop the temporary rolling mean column
    

In [122]:
missing_indexes_far_price = data[data['far_price'].isnull()].index
missing_indexes_near_price = data[data['near_price'].isnull()].index
missing_indexes_far_price, missing_indexes_near_price

(Index([      0,       1,       2,       3,       4,       5,       6,       7,
              8,       9,
        ...
        5232970, 5232971, 5232972, 5232973, 5232974, 5232975, 5232976, 5232977,
        5232978, 5232979],
       dtype='int64', length=2855173),
 Index([      0,       1,       2,       3,       4,       5,       6,       7,
              8,       9,
        ...
        5232970, 5232971, 5232972, 5232973, 5232974, 5232975, 5232976, 5232977,
        5232978, 5232979],
       dtype='int64', length=2855160))

In [123]:
print(data.isnull().sum()) # Checking that all missing elements are no longer empty

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                   0
imbalance_buy_sell_flag          0
reference_price                  0
matched_size                     0
far_price                  2855173
near_price                 2855160
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
time_id                          0
row_id                           0
dtype: int64


In [124]:
numeric_columns = data.select_dtypes(include=['int64', 'float64'])
numeric_columns_array = np.array(numeric_columns.columns)

In [125]:
correlation_far_price = []
correlation_near_price = []
for i in range(1,len(numeric_columns.columns)):
    if numeric_columns_array[i] != 'far_price':
        correlation_far_price_value = numeric_columns['far_price'].corr(numeric_columns[numeric_columns_array[i]])
        correlation_far_price.append(correlation_far_price_value)
    if numeric_columns_array[i] != 'near_price':
        correlation_near_price_value = numeric_columns['near_price'].corr(numeric_columns[numeric_columns_array[i]])
        correlation_near_price.append(correlation_near_price_value)
print(correlation_far_price, correlation_near_price)

[0.0151027333128346, -0.0006998971027982878, 0.026112666117497425, 0.014119148900985827, 0.014444845279485433, 0.01442420940195351, -0.001758626302321915] [0.47356739100587086, -0.0027251165518789875, 0.026112666117497425, 0.4583473297183302, 0.460358814669452, 0.4625678010675628, 0.006031796438557888]


In [126]:
max_correlation_index_far_price = correlation_far_price.index(max(correlation_far_price))
max_correlation_index_near_price = correlation_near_price.index(max(correlation_near_price))
print('Highest correlated column with far_price column is',numeric_columns_array[max_correlation_index_far_price], 'with value:', max(correlation_far_price))
print('Highest correlated column with near_price column is',numeric_columns_array[max_correlation_index_near_price], 'with value:', max(correlation_near_price))

Highest correlated column with far_price column is matched_size with value: 0.026112666117497425
Highest correlated column with near_price column is imbalance_size with value: 0.47356739100587086


- far_price correlation with matched_size 
- near_price correlation with imbalance_size 

In [127]:
data.set_index('matched_size', inplace=True)
data['far_price'] = data['far_price'].interpolate(method='linear', limit_direction='both')
data.reset_index(inplace=True)

data.set_index('imbalance_size', inplace=True)
data['near_price'] = data['near_price'].interpolate(method='linear', limit_direction='both')
data.reset_index(inplace=True)

In [128]:
print(data.isnull().sum()) # Checking that all missing elements are no longer empty

imbalance_size             0
matched_size               0
stock_id                   0
date_id                    0
seconds_in_bucket          0
imbalance_buy_sell_flag    0
reference_price            0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
row_id                     0
dtype: int64
